<a href="https://colab.research.google.com/github/ericburdett/cs673-personal-tutor/blob/master/Personal_Tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Personal Tutor

This notebook contains code for the Personal Tutor System built for CS673: Computational Creativity.


## Imports and Setup

Installing transformers requires the Runtime to be restarted on Colab. The os.kill command does that for us automatically. Just note that Colab will show an error indicating that the runtime has crashed... Ignore it and run the following code blocks

In [0]:
!pip install transformers
!python -m spacy download en_core_web_md
import os
os.kill(os.getpid(), 9) # This will automatically restart the runtime. Colab will show an error... but it works

In [0]:
import torch
import torch.nn.functional as F
import pdb
import string
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import spacy
import numpy as np
import pandas as pd

## Word Distribution

In [0]:
# Download the simple word distribution from GitHub
!wget -O word_dist_full.csv https://raw.githubusercontent.com/ericburdett/cs673-personal-tutor/master/data/word_dist_full.csv

In [0]:
class WordDist():
  def __init__(self):
    self.df = pd.read_csv('word_dist_full.csv', header=None, names=['word', 'freq'])
  
  def getdf(self):
    return self.df

  def dict_normalized(self): 
    copy = self.df.copy()
    copy['freq'] = copy['freq'] / copy['freq'].max()

    return copy.set_index('word').to_dict()['freq']

  def __getitem__(self, index):
    return self.df['word'][index], self.df['freq'][index]

  def __len__(self):
    return len(self.df)

## Language Model and Evaluation Classes


In [0]:
class LanguageModel():
  def __init__(self, mask=None, k=50):
    self.model = GPT2LMHeadModel.from_pretrained('distilgpt2').cuda()
    self.tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
    self.k = k

    if mask == None:
      self.mask = torch.ones(len(self.tokenizer.get_vocab())).cuda()
    else:
      self.mask = torch.tensor(mask).cuda()

  def top_k_logits(self, logits):
    if self.k == 0:
        return logits
    values, _ = torch.topk(logits, self.k)
    min_values = values[-1]
    return torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * -1e10, logits)
  
  def tokenizer(self):
    return self.tokenizer

  def set_mask(self, mask):
    self.mask = torch.tensor(mask).cuda()

  def get_sentences(self, prompt, sentence_length, num_sentences):
    sentences = []

    for i in range(num_sentences):
      sentence = self.get_sentence(prompt, sentence_length)
      sentences.append(sentence)

    return sentences

  def get_sentence(self, prompt, length):
    generated = self.tokenizer.encode(prompt)
    context = torch.tensor([generated]).cuda()

    past = None

    for i in range(length):
      output, past = self.model(context, past=past)
      
      logits = output[..., -1, :].squeeze()
      logits = logits * self.mask # Apply the mask to the logits

      topk_logits = self.top_k_logits(logits)
      topk_log_probs = F.softmax(topk_logits, dim=-1)
      token = torch.multinomial(topk_log_probs, num_samples=1)

      generated += [token.item()]
      context = token.unsqueeze(0)
    
    sequence = self.tokenizer.decode(generated)

    end_index = len(prompt.split('. '))

    return ".".join(sequence.split('.')[0:end_index]) + '.'

In [0]:
class Evaluator():
  def __init__(self, topic):
    self.topic_doc = NLP(topic)

  def get_keywords(self, sentence_doc):
    # Find Nouns and Adjectives
    keywords = []
    for token in sentence_doc:
      pos = token.pos
      if pos in [92, 96]: # NOUN, PNOUN, ADJ , 84
         keywords.append(token)
    
    return keywords

  def get_random_pairs(self, arr, size):
    pairs = []

    try:
      for i in range(size):
        pair = np.random.choice(arr, size=2, replace=False)
        pairs.append(pair)
    except:
      return None

    return pairs

  def generate_scores_array(self, size):
    step = size // 10 + 1
    return np.array([i for i in range(11) for _ in range(step)])[::-1][:size]
  
  def get_score(self, sentences, eval_func, negate=False):
    lengths = [eval_func(sentence) for sentence in sentences]
    sort_indices = np.argsort(np.negative(lengths)) if negate else np.argsort(lengths)
    scores = self.generate_scores_array(len(sentences))
    np.put(scores, sort_indices, scores)
    return scores

  def sentence_length_score(self, sentences):
    return self.get_score(sentences, len, negate=False)

  def topic_score(self, sentences):
    return self.get_score(sentences, self.single_topic_score, negate=True)

  def related_score(self, sentences):
    return self.get_score(sentences, self.single_related_score, negate=True)

  def score_sentences(self, sentences):
    nlp_sentences = [NLP(sentence) for sentence in sentences]

    scores = []
    scores.append(self.sentence_length_score(sentences))
    scores.append(self.topic_score(nlp_sentences))
    scores.append(self.related_score(nlp_sentences))
    # Append scores here for more tests

    return np.mean(scores, axis=0)

  def single_topic_score(self, sentence_doc):
    keywords = self.get_keywords(sentence_doc)
    if len(keywords) < 2:
      return 0

    similarities = []
    for keyword in keywords:
      if keyword.vector_norm:
        similarity = self.topic_doc.similarity(keyword)
      else:
        similarity = 0

      similarities.append(similarity)

    return np.mean(similarities)
  
  def single_related_score(self, sentence_doc):
    keywords = self.get_keywords(sentence_doc)

    # Sample Random Pairs
    pairs = self.get_random_pairs(keywords, 10) # 10 seems like a good number for now...
    if pairs == None or len(pairs) == 0:
      return 0

    if len(keywords) != len(set(keywords)): # If the list contains duplicates, give poor score
      return 0

    # Check Similarity
    similarities = []
    for pair in pairs:

      if pair[0].vector_norm and pair[1].vector_norm:
        similarity = pair[0].similarity(pair[1])
      else:
        similarity = 0

      if similarity >= 1: # Do not give a high similarity score if we are comparing a word with itself
        similarity = 0

      similarities.append(similarity)
      # print('Comparing {} with {}, score: {:.4f}'.format(pair[0], pair[1], similarity))
    
    # print(similarities)

    return np.mean(similarities)

In [0]:
def remove_prompt(sentence, prompt):
  new_sentence = sentence.split(prompt)
  if len(new_sentence) < 2:
    return ''
  else:
    return sentence.split(prompt)[1]

## User Knowledge

In [0]:
class WordMemory:
  def __init__(self, rank, memory_size=10):
    self.rank = rank
    self.memory_size = memory_size
    self.memory = np.zeros(memory_size).tolist()

  def add(self, is_known):
    assert type(is_known) is bool # For now, only allow bools to be added to memory

    self.memory.append(int(is_known))
    if len(self.memory) > self.memory_size: # We are only remembering the past 'memory_size' times we've seen this word 
      self.memory.pop(0)

  def ratio(self):
    if len(self.memory) == 0:
      return -1 # indicates the word has never been seen by the user
    else:
      return np.mean(self.memory)

  def rank(self):
    return self.rank

In [0]:
class UserKnowledge:
  def __init__(self, vocabulary_list, tokenizer, window=500, lower_bound=.8, upper_bound=1.):
    # contains dictionary of word : WordMemoryObject
    self.knowledge = dict([(word, WordMemory(rank=index)) for index, word in enumerate(vocabulary_list)])
    self.ranking = dict([(index, word) for index, word in enumerate(vocabulary_list)])
    self.tokenizer = tokenizer
    self.tokenizer_vocab = tokenizer.get_vocab()

    self.window = window
    self.lower_bound = lower_bound
    self.upper_bound = upper_bound

  def update(self, words, knowns):
    for word, known in zip(words, knowns):
      if word in self.knowledge: # Check first if the word exists in the dictionary
        self.knowledge[word].add(known)
      # else: ## TODO ##
      #   Possibly add it to the knowledge base and then give it more likelihood to be shown again...
  
  def compute_ratios(self): # Known/Total
    return dict([(word, memory.ratio()) for word, memory in self.knowledge.items()])
  
  def compute_mask(self):
    mask = np.ones(len(self.tokenizer_vocab))
    indices = []
    current_num = 0

    # Give higher probability to punctuation so we don't end up with really long sentences
    punc_indices = []
    punc_indices.append(self.tokenizer_vocab[self.tokenizer.tokenize(".")[0]])
    # punc_indices.append(self.tokenizer_vocab[self.tokenizer.tokenize(",")[0]])
    punc_indices.append(self.tokenizer_vocab[self.tokenizer.tokenize("!")[0]])
    # punc_indices.append(self.tokenizer_vocab[self.tokenizer.tokenize(";")[0]])

    for mask_index in punc_indices:
      mask[mask_index] = self.lower_bound - .05

    # Iterate through the rankings in order...
    for index, word in self.ranking.items():
      # if current_num <= self.window:
        # print(word, ',', current_num)

      # Get tokens for the given word with a space pre-pended and also with the word capitalized
      tokens_low = self.tokenizer.tokenize(' ' + str(word))
      tokens_up = self.tokenizer.tokenize(' ' + str(word).capitalize())

      tokens = np.concatenate((tokens_low, tokens_up))
      tokens = list(set(tokens)) # Give us only the unique tokens

      mask_indices = []
      for token in tokens:
        mask_indices.append(self.tokenizer_vocab[token])

      # if the user hasn't learned a given word, give it a higher probability for the language model to produce it
      # if we still need to fill the window, try the most frequent words first
      # as the user learns the easier words, the more difficult words will become more probable
      ratio = self.knowledge[word].ratio()
      if ratio != 1.0 and current_num <= self.window:
        mask_value = self.lower_bound
        current_num += 1
      else:
        mask_value = self.upper_bound

      for mask_index in mask_indices:
        if (mask_value != 1):
          indices.append(mask_index)
          mask[mask_index] = mask_value

    return mask

## Examples

In [0]:
# Pick your topic
# Set up objects for system
TOPIC = 'News'

NLP = spacy.load('en_core_web_md')
LM = LanguageModel(k=25)
EVAL = Evaluator(TOPIC)

### Basic Model without UserKnowledge

In [297]:
# Main App Loop
NUM_SENTENCES = 10
MAX_SENTENCE_LENGTH = 40

LM.set_mask(np.ones(len(LM.tokenizer.get_vocab())))

prompts = ['President Trump said']
prompts_truncate = [False]

rand_index = np.random.randint(len(prompts))
prompt = prompts[rand_index]
should_truncate = prompts_truncate[rand_index]

sentences = LM.get_sentences(prompt, MAX_SENTENCE_LENGTH, NUM_SENTENCES)
if should_truncate:
  sentences = [remove_prompt(sentence, prompt) for sentence in sentences]

scores = EVAL.score_sentences(sentences)
print('Produced Sentence: ', sentences[np.argmax(scores)])

Produced Sentence:  President Trump said this week that his administration was committed to ensuring that women get the healthcare they want.


### Using UserKnowledge to Modify Probability Distribution

In [305]:
# Main App Loop
NUM_SENTENCES = 10
MAX_SENTENCE_LENGTH = 40

user_knowledge = UserKnowledge(WordDist().dict_normalized(), LM.tokenizer)
# Use user_knowledge.update() to update known words -- accepts list of words and list of bools indicating known/unknown
mask = user_knowledge.compute_mask() # Run this function to compute the mask
LM.set_mask(mask) # Set the mask to the language model

prompts = ['President Trump said']
prompts_truncate = [False]

rand_index = np.random.randint(len(prompts))
prompt = prompts[rand_index]
should_truncate = prompts_truncate[rand_index]

sentences = LM.get_sentences(prompt, MAX_SENTENCE_LENGTH, NUM_SENTENCES)
if should_truncate:
  sentences = [remove_prompt(sentence, prompt) for sentence in sentences]

scores = EVAL.score_sentences(sentences)
print('Produced Sentence: ', sentences[np.argmax(scores)])

Produced Sentence:  President Trump said during a news visit this week that he will not be in office.


### Example of updating user_knowledge object and computing mask

In [0]:
# example of updating user_knowledge
user_knowledge = UserKnowledge(WordDist().dict_normalized(), LM.tokenizer)
user_knowledge.update(['the', 'the', 'a', 'from'], [True, False, True, True])
mask = user_knowledge.compute_mask()
# LM.set_mask(mask) # We can update the language model's mask with the set_mask method

In [0]:
# Tokens that we modified that have a higher probability of being shown
tokenizer.convert_ids_to_tokens(np.squeeze(np.where(mask != 1)))

# Old Code

## Imports

In [0]:
!pip install gpt-2-simple
!pip install gtts

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import string
import gzip
import tarfile
from PIL import Image, ImageOps
import gc
import pdb
import pandas as pd
import gpt_2_simple as gpt2
import requests
import tensorflow as tf
import os
from gtts import gTTS 
from IPython.core.ultratb import AutoFormattedTB
from IPython.display import Audio, HTML
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "Request a GPU from Runtime > Change Runtime"

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# Download the children's book corpus from GitHub
!wget -O wiki_simple.txt https://raw.githubusercontent.com/ericburdett/cs673-personal-tutor/master/data/wiki_simple.txt

--2020-02-20 18:26:35--  https://raw.githubusercontent.com/ericburdett/cs673-personal-tutor/master/data/wiki_simple.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47667422 (45M) [text/plain]
Saving to: ‘wiki_simple.txt’

wiki_simple.txt     100%[===================>]  45.46M   199MB/s    in 0.2s    

2020-02-20 18:26:36 (199 MB/s) - ‘wiki_simple.txt’ saved [47667422/47667422]



## Classes

In [0]:
# Class that deals with training and generating text from GPT2
class LanguageModel():
  def __init__(self, model='124M', genre='children', train_steps=200, max_length=150):
    self.download_model(model)
    self.genre = genre
    self.max_length = max_length

    tf.reset_default_graph()
    self.sess = gpt2.start_tf_sess()

    if genre == 'children':
      gpt2.finetune(self.sess, 'wiki_simple.txt', model_name=model, steps=train_steps)
    else:
      raise('The specified genre does not exist')

  # Returns a list of sample texts with a given prefix and suffix
  def generate_text(self, prefix='<|startoftext|>', suffix='.', include_prefix=False, nsamples=5):
    if nsamples < 1 or nsamples > 20:
      raise('Error: nsamples must be within the range 1 <= x <= 20')

    return gpt2.generate(self.sess, prefix=prefix, truncate=suffix, include_prefix=include_prefix, batch_size=nsamples, nsamples=nsamples, return_as_list=True, length=self.max_length)
  
  def download_model(self, model_name):
    if not os.path.isdir(os.path.join("models", model_name)):
      print(f"Downloading {model_name} model...")
      gpt2.download_gpt2(model_name=model_name)
    else:
      print(f"{model_name} model is already downloaded")

In [0]:
# A class that contains some knowledge that the user has acquired over time.
# For example, it may hold the words that the user knows (and how well the user knows them)
class UserKnowledge():
  def __init__(self):
    pass

  # We will likely need some place to store the knowledge we acquire about the user
  # so that we can access it from session to session
  def save_knowledge(self, path):
    pass

In [0]:
# Class that evaluates sentences based on what the system knows about the user
class SentenceEvaluator():
  def __init__(self, level='beginner', user_knowledge=None):
    self.level = 'beginner'
    self.word_dist = WordDist().dict_normalized()
    self.word_dist_threshold = 0.033
    self.word_dist_threshold_step = 0.005
    self.word_dist_difficulty_threshold = 7

    if user_knowledge == None:
      self.user_knowledge = UserKnowledge()
    else:
      self.user_knowledge = user_knowledge

  # We will likely want some method to update the user_knowledge in the evaluator
  # Maybe, we will only pass user_knowledge into the evaluate function?...
  def update_user_knowledge(user_knowledge):
    pass

  # Score the sentences and return the sentence with the highest score
  def evaluate(self, sentences):
    scores = self.score(sentences)
    high_score_index = np.argmax(scores)

    return sentences[high_score_index]

  # Score each sentence based on some criteria
  def score(self, sentences):
    scores = []
    for sentence in sentences:
      score = 0
      score += self.length_score(sentence)
      score += self.word_difficulty(sentence)

      # add other criteria for scoring
      # ...
      # ...
      scores.append(score)
    
    return scores

  # For beginners, we want to favor shorter sentences
  # This method should change as we increase difficulty level
  def length_score(self, sentence):
    length = len(sentence)

    if self.level == 'beginner':
      if length > 0 and length <= 15:
        return 6
      elif length > 15 and length <= 25:
        return 10
      elif length > 25 and length <= 35:
        return 7
      elif length > 35 and length <= 45:
        return 3
      elif length > 45 and length <= 55:
        return 1
      else:
        return 0
    else:
      raise('support for non-beginners is not supported')

  # For beginners, easier the better!
  # This method should change as we increase difficulty level
  def word_difficulty(self, sentence):
    word_scores = []

    for word in sentence.split(' '):
      word = word.lower()
      word_score = self.word_dist.get(word, 0) # Return the word or 0 if it doesn't exist

      if word_score >= self.word_dist_threshold:
        word_scores.append(10)
      elif word_score >= self.word_dist_threshold - self.word_dist_threshold_step:
        word_scores.append(8)
      elif word_score >= self.word_dist_threshold - (2 * self.word_dist_threshold_step):
        word_scores.append(6)
      elif word_score >= self.word_dist_threshold - (3 * self.word_dist_threshold_step):
        word_scores.append(4)
      elif word_score >= self.word_dist_threshold - (4 * self.word_dist_threshold_step):
        word_scores.append(2)
      else:
        word_scores.append(0)

    score_med = np.median(word_scores)

    if score_med >= self.word_dist_difficulty_threshold:
      return 10
    elif score_med >= self.word_dist_difficulty_threshold - 1:
      return 8
    elif score_med >= self.word_dist_difficulty_threshold - 2:
      return 6
    elif score_med >= self.word_dist_difficulty_threshold - 3:
      return 4
    elif score_med >= self.word_dist_difficulty_threshold - 4:
      return 2
    else:
      return 0

In [0]:
class SentenceGenerator():
  def __init__(self, language_model=None, evaluator=None):
    if language_model == None:
      self.language_model = LanguageModel()
    else:
      self.language_model = language_model
    if evaluator == None:
      self.evaluator = SentenceEvaluator()
    else:
      self.evaluator = evaluator

  # Generate a sentence, pick the best one based on evaluation, return the sentence
  def generate(self, print_all_sentences=False):
    # Determine the prefix/suffix based on some kind of criteria that is learned over time
    prefix = self.determine_prefix()
    if prefix == '<|startoftext|>':
      include_prefix = False
    else:
      include_prefix = True
    suffix = self.determine_suffix()

    sentences = self.language_model.generate_text(prefix=prefix, suffix=suffix, include_prefix=include_prefix, nsamples=15)
    sentences = self.filter_punctuation(sentences)
    best_sentence = self.evaluator.evaluate(sentences)

    if print_all_sentences:
      for sentence in sentences:
        print(sentence)

    return best_sentence
  
  # Used to filter unwanted punctuation GPT2 might produce, like newlines
  def filter_punctuation(self, sentences):
    filtered_sentences = []

    for sentence in sentences:
      new_sentence = sentence.replace('\n', ' ')
      new_sentence = new_sentence.translate(str.maketrans('', '', string.punctuation))
      filtered_sentences.append(new_sentence)

    return filtered_sentences

  def determine_prefix(self):
    # Good simple sentence starters...
    # starters = ['<|startoftext|>']
    starters = ['I', 'You', 'The', 'They', 'It', '<|startoftext|>', 'He', 'She', 'My']
    random_index = np.random.randint(0, len(starters)) 

    return starters[random_index]

  def determine_suffix(self):
    return '.'

## Tutoring System

In [0]:
model = LanguageModel('117M', train_steps=200) # Will fine-tune model everytime this is called! -- Will need to be fixed at some point

In [0]:
generator = SentenceGenerator(language_model=model)
best_sentence = generator.generate(print_all_sentences=True)
print("Best Sentence: ", best_sentence)

My own experience with it was that it was a very funny and funny movie 
My game is an open world game 
My second favorite place to eat is at a nearby lake 
My name is Washington SmootHart  and I am an agent of the United Nations 
Myrious s talk with the King of France was not well received and the book was banned 
My wife and her brother were at home when a sudden  thunderbolt  struck the house 
My money was full of things that are not in the movie and were not intended for audience or to be seen by children 
My favorite game is High Roller Ballet 
My own life was spent in the city and in the West Bank  and belonged to the family of the people who lived there 
My hair is round and it looks like the shaft of a gun 
My way was to go to a village called Namur in Afghanistan in 1959 
My students were supposed to be students at the University of East Anglia  but they were supposed to be studying in the department of English 
My statutes were changed to go with the Dukes of France  and the D

In [0]:
def print_options():
  print('0: I don\'t know what this means.')
  print('1: Choose words I don\'t know.')
  print('2: Generate a better sentence.')
  print('3: I need definitions.')
  print('4: I understand! Give me another!')
  print('5: Exit: I\'ve learned enough for today.')

## Text-to-Speech

In [0]:
speech = gTTS(text = best_sentence, lang = 'en', slow = False)
speech.save('speech.mp3')
Audio(filename='speech.mp3', autoplay=True)

In [0]:
Audio('Hedidnotlikethesoundofit.mp3', autoplay=True)

something


## Learn-A-Language Loop
* Terrible Name...
* We need to come up with something!

In [0]:
print("Learn-A-Language - English")

while True:
  print('\nGenerating personalized sentence... Please Wait.')
  sentence = generator.generate()
  print("\nTry this sentence:")
  print(sentence, '\n')
  speech = gTTS(text=sentence, lang='en', slow=False)
  filename = sentence.replace(' ', '') + '.mp3'
  speech.save(filename)
  

  while True:
    print_options()
    Audio(filename=filename, autoplay=False)
    code = input('Enter a code from above:')
    if code in ['0','1','2','3','4','5']:
      code = int(code)
      break
    print('')
  
  if code == 0:
    print('\nI\'m Sorry! This is as get as it gets...')
  elif code == 1:
    print('\nI\'m Sorry! This functionality isn\'t currently available.')
  elif code == 2:
    print('\nNew sentence coming right up!')
  elif code == 3:
    print('\nI\'m Sorry! This functionality isn\'t currently available.')
  elif code == 4:
    print('\nGreat Job! Here\'s another.')
  else:
    print('\nThanks for using Learn-A-Language! Play again soon!')
    break

Learn-A-Language - English

Generating personalized sentence... Please Wait.

Try this sentence:
He did not like the sound of it  

0: I don't know what this means.
1: Choose words I don't know.
2: Generate a better sentence.
3: I need definitions.
4: I understand! Give me another!
5: Exit: I've learned enough for today.


KeyboardInterrupt: ignored

In [0]:
print("Learn-A-Language - English")

while True:
  print('\nGenerating personalized sentence... Please Wait.')
  sentence = generator.generate()
  print("\nTry this sentence:")
  print(sentence, '\n')
  speech = gTTS(text=sentence, lang='en', slow=False)
  filename = sentence.replace(' ', '') + '.mp3'
  speech.save(filename)
  Audio(filename=filename, autoplay=False)

  while True:
    print_options()
    Audio(filename=filename, autoplay=False)
    code = input('Enter a code from above:')
    if code in ['0','1','2','3','4','5']:
      code = int(code)
      break
    print('')
  
  if code == 0:
    print('\nI\'m Sorry! This is as good as it gets...')
  elif code == 1:
    print('\nI\'m Sorry! This functionality isn\'t currently available.')
  elif code == 2:
    print('\nNew sentence coming right up!')
  elif code == 3:
    print('\nI\'m Sorry! This functionality isn\'t currently available.')
  elif code == 4:
    print('\nGreat Job! Here\'s another.')
  else:
    print('\nThanks for using Learn-A-Language! Play again soon!')
    break

Learn-A-Language - English

Generating personalized sentence... Please Wait.

Try this sentence:
He did not like the sound of it  

0: I don't know what this means.
1: Choose words I don't know.
2: Generate a better sentence.
3: I need definitions.
4: I understand! Give me another!
5: Exit: I've learned enough for today.


KeyboardInterrupt: ignored